# Homework 3

In this homework, you will test various time series classification methods. You must choose **three** datasets from [the UCR/UEA time series repository](http://timeseriesclassification.com) and perform the tasks by evaluating the models on three selected datasets.

Note that the questions are not 100% the same as the lab tasks. Please carefully read all descriptions. Compared to previous homework, there is no fixed answer, and we will evaluate your assignment based on your trials rather than the results. When the description does not specify or restrict things during your implementation process, you can choose your way freely. We will always grade based on the written description on each task only.


### Task 0: Preparation

You need to choose **three** datasets from the UCR/UEA time series repository. Please be careful since some UCR datasets can take a long time to be processed - You do not need to choose heavy datasets since it would slow your training/testing process. Use sktime's `load_UCR_UEA_dataset` function to perform. Please note that **you should use each dataset's original train/test splits** to train and report the test scores.

In [ ]:
!pip install sktime -q
!pip install sktime[all_extras] -q
!pip install sktime-dl -q 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.5 MB/s eta

In [ ]:
from sktime.datasets import load_UCR_UEA_dataset
from sktime.datasets import load_italy_power_demand
from sktime.utils.slope_and_trend import _slope

In [ ]:
ECG200_Train = load_UCR_UEA_dataset('ECG200', split='train', return_type='numpy2D')
ECG200_Test = load_UCR_UEA_dataset('ECG200', split='test', return_type='numpy2D')

Earthquake_Train = load_UCR_UEA_dataset('Earthquakes', split='train', return_type='numpy2D')
Earthquake_Test = load_UCR_UEA_dataset('Earthquakes', split='test', return_type='numpy2D')


GunPoint_Train = load_UCR_UEA_dataset('GunPoint', split='train', return_type='numpy2D')
GunPoint_Test = load_UCR_UEA_dataset('GunPoint', split='test', return_type='numpy2D')


In [ ]:
GunPoint_Train = GunPoint_Train[0], GunPoint_Train[1].astype(int) - 1
GunPoint_Test = GunPoint_Test[0], GunPoint_Test[1].astype(int) - 1

### Task 1: Time series classification using deep learning 1



Time series classification problems can be solved using networks like CNN, RNN, or FCN. You can even try to merge different networks. In this task, you must test your three chosen datasets on four other models.

Try to implement four different models: 1) Fully connected network, 2) **[Bi-directional RNN](https://keras.io/api/layers/recurrent_layers/bidirectional/)**, 3) 1D-CNN only, and 4) 1D-CNN + Any RNN variant. Note that you can always link the network to a fully connected layer to match the output size. You can freely construct any structure you want. Report the average test scores of four models on three datasets you chose. It would be four scores in total. Mark the best model in terms of the average test score. Briefly explain the structure you constructed. There is no definitive answer, and it is up to your own model. Here it would be best if you keep the following rules:

- When initially loading the dataset, use sktime's `load_UCR_UEA_dataset` function. This is for our grading purpose.
- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should use Tensorflow's data API (`tf.data`) to manage your dataset and use `shuffle`, `batch,` and `prefetch` functions. This means that you need to convert the data format using the `from_tensor_slices` function. This also means that you need to create your own validation set. You are not limited to using any methods to do this, but you may also need to shuffle the dataset before (for that, you can use `np.random.permutation`).
- You need to clearly report the **test accuracy** of the four models. Training and validation accuracy scores are not enough.
- You may need to deal with datasets of different sizes. In this case, it might be helpful to make a function to create a model that can receive different input sizes as a parameter.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Bidirectional, Flatten, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import train_test_split

import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
RANDOM_STATE = 12345

In [ ]:
# Set random seeds for reproducibility
import random
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
Ecg_train, Ecg_val, Ecg_ytest, Ecg_valtest = train_test_split(ECG200_Train[0],ECG200_Train[1], test_size=0.1,random_state=seed_value)

In [ ]:
Eq_train, Eq_val, Eq_ytest, Eq_valtest = train_test_split(Earthquake_Train[0],Earthquake_Train[1], test_size=0.1,random_state=seed_value)

In [ ]:
Gp_train, Gp_val, Gp_ytest, Gp_valtest = train_test_split(GunPoint_Train[0],GunPoint_Train[1], test_size=0.1,random_state=seed_value)

In [ ]:
def preprocess_data(X, y):
    # Convert to TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    # Shuffle and batch the data
    dataset = dataset.shuffle(buffer_size=len(X))
    dataset = dataset.batch(batch_size=16)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


def load_data(X_train, y_train, X_test=None, y_test=None, val=True):
  y_train_binary = to_categorical(y_train)
  train_dataset = preprocess_data(X_train, y_train_binary)
  if val:
    y_test_binary = to_categorical(y_test)
    val_dataset = preprocess_data(X_test, y_test_binary)
    return train_dataset, val_dataset, X_train
  return train_dataset





In [ ]:
def load_data_rnn(X_train, y_train, X_test=None, y_test=None, timestep=1, val=True):
  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],timestep)
  y_train_binary = to_categorical(y_train)
  train_dataset = preprocess_data(X_train, y_train_binary)
  if val:
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],timestep)
    y_test_binary = to_categorical(y_test)
    val_dataset = preprocess_data(X_test, y_test_binary)
    return train_dataset, val_dataset, X_train
  return train_dataset

#Fully Connected Network

In [ ]:
def create_fc_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Flatten()(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('FCN.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = create_fc_model(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc


In [ ]:
train_dataset, val_dataset, inputs = load_data(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 5s 108ms/step - loss: 0.1315 - accuracy: 0.9667 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 7.0201e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 6.7937e-05 - accuracy: 1.0000
Test accuracy: 1.0


In [ ]:
train_dataset, val_dataset, inputs = load_data(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 2s 21ms/step - loss: 0.5959 - accuracy: 0.7785 - val_loss: 0.4254 - val_accuracy: 0.8182
Epoch 2/100
19/19 [==============================] - 0s 6ms/step - loss: 0.3836 - accuracy: 0.8304 - val_loss: 0.5437 - val_accuracy: 0.5758
Epoch 3/100
19/19 [==============================] - 0s 9ms/step - loss: 0.1266 - accuracy: 0.9481 - val_loss: 0.5498 - val_accuracy: 0.7273
Epoch 4/100
19/19 [==============================] - 0s 9ms/step - loss: 0.0211 - accuracy: 0.9965 - val_loss: 0.8555 - val_accuracy: 0.7273
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.9681 - val_accuracy: 0.6970
Epoch 6/100
9/9 [==============================] - 0s 3ms/step - loss: 0.6289 - accuracy: 0.7338
Test accuracy: 0.7338129281997681


In [ ]:
train_dataset, val_dataset, inputs = load_data(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 2s 170ms/step - loss: 0.7457 - accuracy: 0.5111 - val_loss: 0.6687 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 26ms/step - loss: 0.6921 - accuracy: 0.5778 - val_loss: 0.9217 - val_accuracy: 0.4000
Epoch 3/100
3/3 [==============================] - 0s 41ms/step - loss: 0.6934 - accuracy: 0.5778 - val_loss: 0.6139 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 0s 34ms/step - loss: 0.4131 - accuracy: 0.8222 - val_loss: 0.4415 - val_accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 43ms/step - loss: 0.4043 - accuracy: 0.8222 - val_loss: 0.4839 - val_accuracy: 0.8000
Epoch 6/100
3/3 [==============================] - 0s 19ms/step - loss: 0.3439 - accuracy: 0.8444 - val_loss: 0.3505 - val_accuracy: 0.8000
Epoch 7/100
3/3 [==============================] - 0s 26ms/step - loss: 0.2746 - accuracy: 0.8889 - val_loss: 0.2438 - val_accuracy: 0.8000
Epoch 8/100
3/3 [==

In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the Fully Connected Network:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the Fully Connected Network: {round(Avg_test_loss,4)}")

Average Test Accuracy of the Fully Connected Network:0.829
Average Test Loss of the Fully Connected Network: 0.3394


#Bi-Directional RNN

In [ ]:
def create_bidirectional_rnn_model(input_shape, num_classes, num_units=64, dropout_rate=0.2):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(GRU(num_units, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True)),
        Bidirectional(GRU(num_units, dropout=dropout_rate, recurrent_dropout=dropout_rate)),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('bidirectional_rnn_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = create_bidirectional_rnn_model(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 18s 892ms/step - loss: 0.1508 - accuracy: 0.8333 - val_loss: 2.2868e-04 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 3s 538ms/step - loss: 4.3587e-06 - accuracy: 1.0000 - val_loss: 3.2804e-04 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 2s 379ms/step - loss: 5.2451e-07 - accuracy: 1.0000 - val_loss: 3.8965e-04 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 2s 374ms/step - loss: 1.5077e-05 - accuracy: 1.0000 - val_loss: 2.7354e-04 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 2s 378ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 4s 729ms/step - loss: 5.2982e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 3s 374ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 61s 3s/step - loss: 0.5069 - accuracy: 0.8235 - val_loss: 0.4247 - val_accuracy: 0.8485
Epoch 2/100
19/19 [==============================] - 66s 4s/step - loss: 0.4614 - accuracy: 0.8166 - val_loss: 0.4140 - val_accuracy: 0.8788
Epoch 3/100
19/19 [==============================] - 49s 3s/step - loss: 0.4498 - accuracy: 0.8270 - val_loss: 0.4320 - val_accuracy: 0.8485
Epoch 4/100
19/19 [==============================] - 54s 3s/step - loss: 0.4382 - accuracy: 0.8304 - val_loss: 0.4615 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 49s 3s/step - loss: 0.4520 - accuracy: 0.8062 - val_loss: 0.4230 - val_accuracy: 0.8485
Epoch 6/100
19/19 [==============================] - 52s 3s/step - loss: 0.4282 - accuracy: 0.8131 - val_loss: 0.4943 - val_accuracy: 0.7576
Epoch 7/100
9/9 [==============================] - 2s 209ms/step - loss: 0.5214 - accuracy: 0.7770
Test accuracy: 0.7769784331321716


In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 16s 1s/step - loss: 1.1354 - accuracy: 0.4889 - val_loss: 0.7559 - val_accuracy: 0.4000
Epoch 2/100
3/3 [==============================] - 2s 601ms/step - loss: 0.7504 - accuracy: 0.4889 - val_loss: 0.6837 - val_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 3s 1s/step - loss: 0.7021 - accuracy: 0.5556 - val_loss: 0.7024 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 4s 1s/step - loss: 0.6924 - accuracy: 0.5111 - val_loss: 0.7206 - val_accuracy: 0.4000
Epoch 5/100
3/3 [==============================] - 4s 1s/step - loss: 0.6996 - accuracy: 0.4889 - val_loss: 0.7310 - val_accuracy: 0.4000
Epoch 6/100
3/3 [==============================] - 2s 678ms/step - loss: 0.7049 - accuracy: 0.4889 - val_loss: 0.7201 - val_accuracy: 0.4000
Epoch 7/100
10/10 [==============================] - 1s 69ms/step - loss: 0.6972 - accuracy: 0.4867
Test accuracy: 0.4866666793823242


In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the Bi-Directional RNN:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the Bi-Directional RNN: {round(Avg_test_loss,4)}")

Average Test Accuracy of the Bi-Directional RNN:0.7545
Average Test Loss of the Bi-Directional RNN: 0.4062


#1D-CNN Only

In [ ]:
def create_1d_cnn_model(input_shape, num_classes, num_filters=64, kernel_size=5, pool_size=2, dropout_rate=0.2):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(num_filters, kernel_size, padding='same', kernel_regularizer=l2(0.01)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(pool_size=pool_size),
        Conv1D(num_filters, kernel_size, padding='same', kernel_regularizer=l2(0.01)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('1d_cnn_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = create_1d_cnn_model(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 2s 65ms/step - loss: 0.9121 - accuracy: 0.8778 - val_loss: 0.7769 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 17ms/step - loss: 0.7974 - accuracy: 1.0000 - val_loss: 0.8090 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 15ms/step - loss: 0.7960 - accuracy: 1.0000 - val_loss: 0.7634 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 15ms/step - loss: 0.7321 - accuracy: 1.0000 - val_loss: 0.6779 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 14ms/step - loss: 0.6402 - accuracy: 1.0000 - val_loss: 0.5804 - val_accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 14ms/step - loss: 0.5433 - accuracy: 1.0000 - val_loss: 0.4867 - val_accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 0s 15ms/step - loss: 0.4534 - accuracy: 1.0000 - val_loss: 0.4035 - val_accuracy: 1.0000
Epoch 8/100
6/6 [===

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 2s 47ms/step - loss: 9.1402 - accuracy: 0.7301 - val_loss: 6.0494 - val_accuracy: 0.3636
Epoch 2/100
19/19 [==============================] - 1s 36ms/step - loss: 2.6754 - accuracy: 0.8097 - val_loss: 1.7566 - val_accuracy: 0.7273
Epoch 3/100
19/19 [==============================] - 1s 36ms/step - loss: 1.4111 - accuracy: 0.8374 - val_loss: 2.9443 - val_accuracy: 0.8485
Epoch 4/100
19/19 [==============================] - 1s 36ms/step - loss: 0.8859 - accuracy: 0.8858 - val_loss: 3.5408 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 1s 63ms/step - loss: 0.9340 - accuracy: 0.8927 - val_loss: 5.6056 - val_accuracy: 0.8485
Epoch 6/100
19/19 [==============================] - 1s 65ms/step - loss: 3.1199 - accuracy: 0.7716 - val_loss: 4.9008 - val_accuracy: 0.8485
Epoch 7/100
9/9 [==============================] - 0s 17ms/step - loss: 2.0321 - accuracy: 0.6403
Test accuracy: 0.6402877569198608


In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 3s 215ms/step - loss: 2.0457 - accuracy: 0.5556 - val_loss: 1.3978 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 33ms/step - loss: 2.3826 - accuracy: 0.6667 - val_loss: 1.5723 - val_accuracy: 0.4000
Epoch 3/100
3/3 [==============================] - 0s 38ms/step - loss: 1.5684 - accuracy: 0.6667 - val_loss: 2.2281 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 0s 37ms/step - loss: 1.3092 - accuracy: 0.8000 - val_loss: 1.1781 - val_accuracy: 0.8000
Epoch 5/100
3/3 [==============================] - 0s 34ms/step - loss: 1.3029 - accuracy: 0.8222 - val_loss: 1.9100 - val_accuracy: 0.4000
Epoch 6/100
3/3 [==============================] - 0s 31ms/step - loss: 1.1592 - accuracy: 0.9111 - val_loss: 1.4592 - val_accuracy: 0.6000
Epoch 7/100
3/3 [==============================] - 0s 31ms/step - loss: 1.0334 - accuracy: 0.8667 - val_loss: 1.0455 - val_accuracy: 0.8000
Epoch 8/100
3/3 [==

In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the 1D-CNN Model:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the 1D-CNN Model: {round(Avg_test_loss,4)}")

Average Test Accuracy of the 1D-CNN Model:0.789
Average Test Loss of the 1D-CNN Model: 0.9704


#1D-CNN + Any RNN variant.

In [ ]:
def create_1d_cnn_lstm_model(input_shape, num_classes, num_filters=64, kernel_size=5, pool_size=2, dropout_rate=0.2):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(num_filters, kernel_size, padding='same', kernel_regularizer=l2(0.01)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(pool_size=pool_size),
        LSTM(num_filters, dropout=dropout_rate, recurrent_dropout=dropout_rate),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('1d_cnn_lstm_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = create_1d_cnn_lstm_model(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 4s 155ms/step - loss: 0.1388 - accuracy: 0.9444 - val_loss: 0.0160 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 68ms/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.0118 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 68ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.0093 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 72ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 74ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 67ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 1s 100ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 8/100
6/6 [=

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 17s 608ms/step - loss: 0.4858 - accuracy: 0.7716 - val_loss: 0.4962 - val_accuracy: 0.8485
Epoch 2/100
19/19 [==============================] - 6s 335ms/step - loss: 0.4316 - accuracy: 0.8097 - val_loss: 0.4407 - val_accuracy: 0.8485
Epoch 3/100
19/19 [==============================] - 9s 478ms/step - loss: 0.4206 - accuracy: 0.8201 - val_loss: 0.4166 - val_accuracy: 0.8485
Epoch 4/100
19/19 [==============================] - 6s 335ms/step - loss: 0.4127 - accuracy: 0.8062 - val_loss: 0.4587 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 11s 570ms/step - loss: 0.4164 - accuracy: 0.8270 - val_loss: 0.5126 - val_accuracy: 0.8485
Epoch 6/100
19/19 [==============================] - 7s 382ms/step - loss: 0.4205 - accuracy: 0.7958 - val_loss: 0.3945 - val_accuracy: 0.8485
Epoch 7/100
19/19 [==============================] - 9s 443ms/step - loss: 0.4171 - accuracy: 0.8131 - val_loss: 0.4588 - val_accuracy: 0.84

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 6s 310ms/step - loss: 0.9943 - accuracy: 0.5111 - val_loss: 0.7044 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 113ms/step - loss: 0.8390 - accuracy: 0.5333 - val_loss: 0.6953 - val_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 0s 110ms/step - loss: 0.7072 - accuracy: 0.4889 - val_loss: 0.6899 - val_accuracy: 0.8000
Epoch 4/100
3/3 [==============================] - 0s 117ms/step - loss: 0.7453 - accuracy: 0.4889 - val_loss: 0.6849 - val_accuracy: 0.6000
Epoch 5/100
3/3 [==============================] - 0s 108ms/step - loss: 0.6727 - accuracy: 0.6000 - val_loss: 0.6845 - val_accuracy: 0.6000
Epoch 6/100
3/3 [==============================] - 0s 118ms/step - loss: 0.6797 - accuracy: 0.6222 - val_loss: 0.6825 - val_accuracy: 0.6000
Epoch 7/100
3/3 [==============================] - 0s 111ms/step - loss: 0.6806 - accuracy: 0.6000 - val_loss: 0.6822 - val_accuracy: 0.6000
Epoch 8/100
3

In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the 1D-CNN+RNN Variant Model:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the 1D-CNN+RNN Variant Model: {round(Avg_test_loss,4)}")

Average Test Accuracy of the 1D-CNN+RNN Variant Model:0.7472
Average Test Loss of the 1D-CNN+RNN Variant Model: 0.4317


### Task 2: Time series classification using deep learning 2

There has been several neural network models dedicated to time series classification. Besides your own models that you developed in Lab 4, now you will develop such dedicated models by referring to some papers, and test if they indeed perform better than your rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Use the same four datasets, and test sktime's `CNNClassifier` and MLP and FCN models you develop. Report test scores of three models (`CNNClassifier`, MLP, and FCN) on three datasets you chose. It would be nine scores in total. For MLP and FCN, you may need to satisfy the following requirement:

- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should run the model at least 10 epochs.
- You can use the same processed datasets in Task 1. For `CNNClassifier`, as you cannot use `tf.Data`, you may put the training set directly.
- For `CNNClassifier`, you can run it with the default parameters or reduce the number of epoch (default is 2000).
- Please use the predefined test dataset to report the test scores.

Note that the main purpose of this task is to check if you can develop a similar network structure with description. If the detail of the specific part (e.g., size of one layer or some custom parameters like epoch) is missing in the paper, you can set it on your own.
 

#Using SKtime's CNNClassifier

In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier
def classify(epochs, batch_size, verbose, Train, Test):
  cnn = CNNClassifier(n_epochs=epochs,batch_size=batch_size,verbose=verbose) 
  cnn.fit(Train[0], Train[1]) 
  preds = cnn.predict(Test[0])
  print(f'Test Accuracy Score:{accuracy_score(Test[1],preds)}')
  return accuracy_score(Test[1],preds)

In [ ]:
ECG_ACC = classify(20,4,False,ECG200_Train, ECG200_Test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


25/25 [==============================] - 0s 2ms/step
Test Accuracy Score:0.8


In [ ]:
EQ_ACC = classify(20,4,False,Earthquake_Train,Earthquake_Test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


35/35 [==============================] - 0s 2ms/step
Test Accuracy Score:0.7482014388489209


In [ ]:
GP_ACC = classify(20,4,False,GunPoint_Train,GunPoint_Test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


38/38 [==============================] - 0s 2ms/step
Test Accuracy Score:0.74


In [ ]:
Avg_test_acc = np.mean([ECG_ACC, EQ_ACC, GP_ACC])
print(f"Average Test Accuracy of the Sktime CNN Classifier:{round(Avg_test_acc,4)}")

Average Test Accuracy of the Sktime CNN Classifier:0.7627


#FCN Classifier

In [ ]:
def FCN_Classifier(input_shape, num_classes):
  input_layer = Input(input_shape)
  conv1 = Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
  conv1 = keras.layers.BatchNormalization()(conv1)
  conv1 = keras.layers.Activation(activation='relu')(conv1)

  conv2 = Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
  conv2 = keras.layers.BatchNormalization()(conv2)
  conv2 = keras.layers.Activation('relu')(conv2)

  conv3 = Conv1D(128, kernel_size=3,padding='same')(conv2)
  conv3 = keras.layers.BatchNormalization()(conv3)
  conv3 = keras.layers.Activation('relu')(conv3)
  
  gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)
  output_layer = keras.layers.Dense(num_classes, activation='softmax')(gap_layer)

  model = Model(inputs=input_layer, outputs=output_layer)
  return model
 


In [ ]:
# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('FCN_Classifier.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = FCN_Classifier(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 3s 204ms/step - loss: 0.2261 - accuracy: 0.8222 - val_loss: 2.6345e-06 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 1s 129ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 1s 123ms/step - loss: 1.3489e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 1s 117ms/step - loss: 3.9791e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 1s 82ms/step - loss: 2.0032e-05 - accuracy: 1.0000 - val_loss: 1.1921e-08 - val_accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 73ms/step - loss: 1.3708e-05 - accuracy: 1.0000 - val_loss: 9.5367e-08 - val_accuracy: 1.0000
Epoch 7/100
7/7 [==============================] - 0s 16ms/step - loss: 2.3842e-09 - accuracy: 1.0000
Test accuracy:

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 9s 322ms/step - loss: 0.5191 - accuracy: 0.7924 - val_loss: 1.8034 - val_accuracy: 0.8485
Epoch 2/100
19/19 [==============================] - 8s 418ms/step - loss: 0.5018 - accuracy: 0.8166 - val_loss: 0.3974 - val_accuracy: 0.8485
Epoch 3/100
19/19 [==============================] - 9s 481ms/step - loss: 0.4958 - accuracy: 0.8131 - val_loss: 0.3879 - val_accuracy: 0.8485
Epoch 4/100
19/19 [==============================] - 7s 347ms/step - loss: 0.4721 - accuracy: 0.8166 - val_loss: 0.4115 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 7s 390ms/step - loss: 0.4524 - accuracy: 0.8166 - val_loss: 0.3981 - val_accuracy: 0.8485
Epoch 6/100
19/19 [==============================] - 5s 285ms/step - loss: 0.4656 - accuracy: 0.8166 - val_loss: 0.3902 - val_accuracy: 0.8485
Epoch 7/100
19/19 [==============================] - 7s 374ms/step - loss: 0.4503 - accuracy: 0.8166 - val_loss: 0.4070 - val_accuracy: 0.8485

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 2s 205ms/step - loss: 0.9263 - accuracy: 0.4889 - val_loss: 2.8275 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 137ms/step - loss: 0.6781 - accuracy: 0.5778 - val_loss: 9.2390 - val_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 1s 178ms/step - loss: 0.6501 - accuracy: 0.4889 - val_loss: 9.1004 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 1s 171ms/step - loss: 0.6299 - accuracy: 0.6667 - val_loss: 6.4959 - val_accuracy: 0.6000
Epoch 5/100
3/3 [==============================] - 1s 176ms/step - loss: 0.6284 - accuracy: 0.6000 - val_loss: 7.3786 - val_accuracy: 0.6000
Epoch 6/100
10/10 [==============================] - 0s 41ms/step - loss: 3.4361 - accuracy: 0.4933
Test accuracy: 0.4933333396911621


In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the FCN Network:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the FCN Network: {round(Avg_test_loss,4)}")

Average Test Accuracy of the FCN Network:0.7472
Average Test Loss of the FCN Network: 1.3313


#MLP Classifier

In [ ]:
def MLP_Classifier(input_shape, num_classes):
  input_layer = Input(input_shape)
  # flatten/reshape because when multivariate all should be on the same axis 
  input_layer_flattened = keras.layers.Flatten()(input_layer)
  layer_1 = Dropout(0.1)(input_layer_flattened)
  layer_1 = Dense(500, activation='relu')(layer_1)
  layer_2 = Dropout(0.2)(layer_1)
  layer_2 = Dense(500, activation='relu')(layer_2)

  layer_3 = Dropout(0.2)(layer_2)
  layer_3 = Dense(500, activation='relu')(layer_3)

  output_layer = Dropout(0.3)(layer_3)
  output_layer = Dense(num_classes, activation='softmax')(output_layer)

  model = Model(inputs=input_layer, outputs=output_layer)
  return model


In [ ]:
# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('MLP_Classifier.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = MLP_Classifier(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 2s 78ms/step - loss: 0.0942 - accuracy: 0.9556 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test accuracy: 1.0


In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 1s 22ms/step - loss: 2.0209 - accuracy: 0.7405 - val_loss: 0.5239 - val_accuracy: 0.8485
Epoch 2/100
19/19 [==============================] - 0s 14ms/step - loss: 0.4644 - accuracy: 0.8097 - val_loss: 0.7281 - val_accuracy: 0.8485
Epoch 3/100
19/19 [==============================] - 0s 13ms/step - loss: 0.5107 - accuracy: 0.8651 - val_loss: 0.5502 - val_accuracy: 0.7879
Epoch 4/100
19/19 [==============================] - 0s 13ms/step - loss: 0.4242 - accuracy: 0.8720 - val_loss: 0.7602 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 0s 13ms/step - loss: 0.3074 - accuracy: 0.9170 - val_loss: 0.7068 - val_accuracy: 0.7879
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5896 - accuracy: 0.7482
Test accuracy: 0.7482014298439026


In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 1s 93ms/step - loss: 14.5846 - accuracy: 0.4000 - val_loss: 5.6780 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 21ms/step - loss: 3.0627 - accuracy: 0.5333 - val_loss: 0.7625 - val_accuracy: 0.4000
Epoch 3/100
3/3 [==============================] - 0s 18ms/step - loss: 0.7811 - accuracy: 0.4667 - val_loss: 0.6266 - val_accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 21ms/step - loss: 0.5837 - accuracy: 0.6889 - val_loss: 0.5382 - val_accuracy: 0.8000
Epoch 5/100
3/3 [==============================] - 0s 19ms/step - loss: 0.4310 - accuracy: 0.8000 - val_loss: 0.2990 - val_accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 21ms/step - loss: 0.4110 - accuracy: 0.8667 - val_loss: 0.4928 - val_accuracy: 0.8000
Epoch 7/100
3/3 [==============================] - 0s 22ms/step - loss: 0.2800 - accuracy: 0.8444 - val_loss: 1.2304 - val_accuracy: 0.6000
Epoch 8/100
3/3 [==

In [ ]:
Avg_test_acc = np.mean([ECG_acc, Eq_acc, Gp_acc])
Avg_test_loss = np.mean([ECG_loss, Eq_loss, Gp_loss])
print(f"Average Test Accuracy of the MLP Classifier:{round(Avg_test_acc,4)}")
print(f"Average Test Loss of the MLP Classifier: {round(Avg_test_loss,4)}")

Average Test Accuracy of the MLP Classifier:0.8205
Average Test Loss of the MLP Classifier: 0.3741


### Task 3: Time series classification using deep learning 3

Next, you can try to further improve your model by selecting one of the following ideas (or you can suggest your own ideas that are not trivial):
- Use residual connection to your model (= implement **ResNet** in [this](https://arxiv.org/abs/1611.06455) paper). 
- Add transformer/attention layer to your model (you can choose any model from Tasks 1 and 2).
- Use RNN and CNN networks separately, create two to three layers individually, and concatenate them. This means that until the third (or second) layer, you have two different networks handling the same dataset, and after that, you concatenate the output and finish with any FCN layer. Check [this post](https://stackoverflow.com/questions/59168306/how-to-combine-lstm-and-cnn-in-timeseries-classification) to get inspired.
- Apply any sktime's transformer (not attention transformer) first to the dataset and run any deep learning model you already developed in Tasks 2 and 3. In this case, you need to choose at least two transformers and apply them together.
- Train the model on multiple similar datasets and test it on one specific test set. Check if the model can be improved if it is trained on multiple datasets. However, for this, you may need to choose the similar datasets based on their classification (UCR repository has a specific dataset type such as **AUDIO** or **MOTION**).

Choose one model you want from the models you have developed in Tasks 1 and 2. Select one idea, try implementing it, and check if you can improve the performance. Note that you do not need to prove that the accuracy scores increase but must explain your trials. Report test scores on three datasets you chose.

#ResNet_Classifier

In [ ]:
def ResNet_Classifier(input_shape, num_classes):
  n_feature_maps = 64
  input_layer = Input(input_shape)
  # BLOCK 1
  conv_x = Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
  conv_x = keras.layers.BatchNormalization()(conv_x)
  conv_x = keras.layers.Activation('relu')(conv_x)

  conv_y = Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
  conv_y = keras.layers.BatchNormalization()(conv_y)
  conv_y = keras.layers.Activation('relu')(conv_y)

  conv_z = Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
  conv_z = keras.layers.BatchNormalization()(conv_z)

  # expand channels for the sum
  shortcut_y = Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
  shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
  output_block_1 = keras.layers.add([shortcut_y, conv_z])
  output_block_1 = keras.layers.Activation('relu')(output_block_1)

  # BLOCK 2
  conv_x = Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
  conv_x = keras.layers.BatchNormalization()(conv_x)
  conv_x = keras.layers.Activation('relu')(conv_x)

  conv_y = Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
  conv_y = keras.layers.BatchNormalization()(conv_y)
  conv_y = keras.layers.Activation('relu')(conv_y)

  conv_z = Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
  conv_z = keras.layers.BatchNormalization()(conv_z)

  # expand channels for the sum
  shortcut_y = Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
  shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

  output_block_2 = keras.layers.add([shortcut_y, conv_z])
  output_block_2 = keras.layers.Activation('relu')(output_block_2)

  # BLOCK 3
  conv_x = Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
  conv_x = keras.layers.BatchNormalization()(conv_x)
  conv_x = keras.layers.Activation('relu')(conv_x)

  conv_y = Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
  conv_y = keras.layers.BatchNormalization()(conv_y)
  conv_y = keras.layers.Activation('relu')(conv_y)

  conv_z = Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
  conv_z = keras.layers.BatchNormalization()(conv_z)

  # no need to expand channels because they are equal
  shortcut_y = keras.layers.BatchNormalization()(output_block_2)
  output_block_3 = keras.layers.add([shortcut_y, conv_z])
  output_block_3 = keras.layers.Activation('relu')(output_block_3)

  # FINAL
  gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
  output_layer = Dense(num_classes, activation='softmax')(gap_layer)

  model = Model(inputs=input_layer, outputs=output_layer)
  return model

In [ ]:
# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('ResNet_Classifier.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = ResNet_Classifier(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Ecg_train, Ecg_ytest, Ecg_val, Ecg_valtest, val=True)
test_dataset = load_data_rnn(ECG200_Test[0],ECG200_Test[1],val=False)

ECG_loss, ECG_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
6/6 [==============================] - 10s 339ms/step - loss: 0.2707 - accuracy: 0.8222 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 1s 138ms/step - loss: 1.1401e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 1s 139ms/step - loss: 1.5875e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 1s 234ms/step - loss: 6.0174e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 1s 239ms/step - loss: 3.4690e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test accuracy: 1.0


In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Eq_train, Eq_ytest, Eq_val, Eq_valtest, val=True)
test_dataset = load_data_rnn(Earthquake_Test[0],Earthquake_Test[1],val=False)

Eq_loss, Eq_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
19/19 [==============================] - 24s 815ms/step - loss: 0.5300 - accuracy: 0.8131 - val_loss: 256.8724 - val_accuracy: 0.1515
Epoch 2/100
19/19 [==============================] - 15s 774ms/step - loss: 0.4753 - accuracy: 0.8062 - val_loss: 74.7428 - val_accuracy: 0.1515
Epoch 3/100
19/19 [==============================] - 14s 687ms/step - loss: 0.5184 - accuracy: 0.7958 - val_loss: 0.8097 - val_accuracy: 0.8485
Epoch 4/100
19/19 [==============================] - 17s 883ms/step - loss: 0.4473 - accuracy: 0.8166 - val_loss: 0.4091 - val_accuracy: 0.8485
Epoch 5/100
19/19 [==============================] - 13s 639ms/step - loss: 0.4427 - accuracy: 0.8166 - val_loss: 1.8276 - val_accuracy: 0.2727
Epoch 6/100
19/19 [==============================] - 14s 733ms/step - loss: 0.4972 - accuracy: 0.7785 - val_loss: 0.4360 - val_accuracy: 0.8485
Epoch 7/100
19/19 [==============================] - 18s 929ms/step - loss: 0.4716 - accuracy: 0.8166 - val_loss: 0.3861 - val_accura

In [ ]:
train_dataset, val_dataset, inputs = load_data_rnn(Gp_train, Gp_ytest, Gp_val, Gp_valtest, val=True)
test_dataset = load_data_rnn(GunPoint_Test[0],GunPoint_Test[1],val=False)

Gp_loss, Gp_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 2, 100)

Epoch 1/100
3/3 [==============================] - 7s 451ms/step - loss: 1.0808 - accuracy: 0.6667 - val_loss: 176.5463 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 1s 244ms/step - loss: 0.8494 - accuracy: 0.6889 - val_loss: 4541.4990 - val_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 1s 359ms/step - loss: 0.6666 - accuracy: 0.6444 - val_loss: 9646.0830 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 1s 354ms/step - loss: 0.5642 - accuracy: 0.7111 - val_loss: 8316.3789 - val_accuracy: 0.6000
Epoch 5/100
3/3 [==============================] - 1s 342ms/step - loss: 0.6180 - accuracy: 0.7333 - val_loss: 4786.0171 - val_accuracy: 0.6000
Epoch 6/100
10/10 [==============================] - 1s 59ms/step - loss: 216.5043 - accuracy: 0.4933
Test accuracy: 0.4933333396911621


### Task 4: Time series classification using sktime

As we saw above, we can use RandomizedSearch to find optimal parameter options. However, it does not support scikit-learn's classifiers such as DecisionTree or RandomForest. However, as we tried, it is also possible to use the output of the sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. Sktime supports this with `SklearnClassifierPipeline` to put a  scikit-learn classifier and sktime's transformer together and you need to implement it.

Pick one classifier from scikit-learn (that can be anything! e.g., Decision Tree or Logistic Regressor) and two transformers from sktime and create `SklearnClassifierPipeline.` As we tried in this lab, that can be **Rocket with RandomForest** or **Catch22 with DecisionTree**. Pick one parameter from each module (in total, three, one from the classifier and two from two transformers) and run a randomized search on the pipeline you define and report the test score of the best model found by the randomized Search. Compare your best score to the score from the same model with the default setting.

Task 4 involves a time-consuming process, so you can only choose **one dataset** to perform the task above. Also, note that you do not need to perform better by conducting a randomized search for this task (but still good to try!).

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.dictionary_based import _paa
from sktime.transformations.series.exponent import ExponentTransformer
from sktime.classification.compose import SklearnClassifierPipeline
from sktime.classification.base import BaseClassifier

#Baseline SklearnClassifierPipeline

In [ ]:
X_train = Ecg_train
X_test = Ecg_val
y_train = Ecg_ytest
y_test = Ecg_valtest

In [ ]:
clf = RandomForestClassifier()
rocket = Rocket(num_kernels=10000)
exp = ExponentTransformer()
pipeline = SklearnClassifierPipeline(clf, [rocket, exp])
pipeline = pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(f'Test Accuracy Score: {accuracy_score(y_test,y_pred)}')

Test Accuracy Score: 0.9


#RandomizedSearchCV for SKlearnClassifierPipeline

In [ ]:
# define the classifier and transformers
clf = RandomForestClassifier()
rocket = Rocket(num_kernels=10000)
exp = ExponentTransformer()

# define the pipeline
pipeline = SklearnClassifierPipeline(clf, [rocket, exp])

# define the parameter grid for randomized search
param_grid = {
    'classifier__n_estimators': [100, 500, 1000],
    'Rocket__num_kernels': [500, 1000, 5000, 10000],
    'ExponentTransformer__power': [0.1, 0.3, 0.5]
}

# define the randomized search
rs = RandomizedSearchCV(
    pipeline, param_grid, cv=5, n_iter=10, random_state=42, n_jobs=-1
)

# fit the randomized search
rs.fit(X_train, y_train)

# get the best estimator from randomized search
best_estimator = rs.best_estimator_

# evaluate the best estimator on test data
y_pred = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(best_estimator)
print("Test accuracy:", accuracy)


SklearnClassifierPipeline(classifier=RandomForestClassifier(n_estimators=1000),
                          transformers=[Rocket(), ExponentTransformer()])
Test accuracy: 0.9


### Task 5: Multivariate time series classification

Time series can be **multivariate**, which means there can be many values (= data points) describing one time point. In this task, you will use one **multivariate** dataset (**BasicMotions**) and try to run one deep learning model and one sktime model to see if those models work well on multivariate time series.

- Use [BasicMotions](https://timeseriesclassification.com/description.php?Dataset=BasicMotions) dataset in the UCR/UEA repository.
- Run **ROCKET** and **the best tensorflow deep learning model from the previous tasks** on BasicMotions. You need to adjust the deep learning model's input layer to handle this multivariate dataset.
- Use sktime's `load_UCR_UEA_dataset` function to perform. You should use each dataset's original train/test splits.
- For the deep learning model, you should transform it using TensorFlow data API (`tf.data`) to manage your dataset and use `shuffle`, `batch`, and `prefetch` functions. This means that you need to create the validation set first. 
- For training, you need to run at least 10 epochs for your deep learning model. For ROCKET, you can keep the default parameter options. 
- Report the test scores of both models on the predefined test set.

In [ ]:
BasicMotions_Train = load_UCR_UEA_dataset('BasicMotions', split='train', return_type='numpy3d')
BasicMotions_Test = load_UCR_UEA_dataset('BasicMotions', split='test', return_type='numpy3d')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [ ]:
BasicMotions_Train = BasicMotions_Train[0], le.fit_transform(BasicMotions_Train[1])
BasicMotions_Test = BasicMotions_Test[0], le.fit_transform(BasicMotions_Test[1])

In [ ]:
BM_train, BM_val, BM_ytest, BM_valtest = train_test_split(BasicMotions_Train[0],BasicMotions_Train[1], test_size = 0.1, random_state=seed_value)

#ROCKET Classifier

In [ ]:
from sktime.classification.kernel_based import RocketClassifier
clf = RocketClassifier(num_kernels=5000) 
clf.fit(BM_train, BM_ytest)
preds = clf.predict(BM_val)
test_preds = clf.predict(BasicMotions_Test[0])
print(f'val_accuracy: {accuracy_score(BM_valtest,preds)}')
print(f'test_accuracy: {accuracy_score(BasicMotions_Test[1],test_preds)}')

val_accuracy: 1.0
test_accuracy: 1.0


#Deep Learning Model

In [ ]:
def create_1d_cnn_lstm_model(input_shape, num_classes, num_filters=64, kernel_size=5, pool_size=2, dropout_rate=0.2):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(num_filters, kernel_size, padding='same', kernel_regularizer=l2(0.01)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(pool_size=pool_size),
        LSTM(num_filters, dropout=dropout_rate, recurrent_dropout=dropout_rate),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('1d_cnn_lstm_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False)

# Create the model
def train_model(train_dataset, val_dataset, test_dataset, X_train, num_classes, epochs, lr=0.01):
  model = create_1d_cnn_lstm_model(input_shape=X_train.shape[1:], num_classes=num_classes)
  optimizer = Adam(learning_rate=lr)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping, model_checkpoint])
  test_loss, test_acc = model.evaluate(test_dataset)
  print('Test accuracy:', test_acc)
  return test_loss, test_acc

In [ ]:
train_dataset, val_dataset, inputs = load_data(BM_train, BM_ytest, BM_val, BM_valtest, val=True)
test_dataset = load_data(BasicMotions_Test[0],BasicMotions_Test[1],val=False)

BM_loss, BM_acc = train_model(train_dataset, val_dataset, test_dataset, inputs, 4, 100)

Epoch 1/100
3/3 [==============================] - 6s 394ms/step - loss: 2.2306 - accuracy: 0.1667 - val_loss: 1.9824 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 26ms/step - loss: 1.5685 - accuracy: 0.5833 - val_loss: 1.9963 - val_accuracy: 0.2500
Epoch 3/100
3/3 [==============================] - 0s 36ms/step - loss: 1.2303 - accuracy: 0.8611 - val_loss: 1.5259 - val_accuracy: 0.7500
Epoch 4/100
3/3 [==============================] - 0s 27ms/step - loss: 0.9878 - accuracy: 0.9167 - val_loss: 1.4231 - val_accuracy: 0.7500
Epoch 5/100
3/3 [==============================] - 0s 31ms/step - loss: 0.7450 - accuracy: 0.9722 - val_loss: 1.3491 - val_accuracy: 0.7500
Epoch 6/100
3/3 [==============================] - 0s 29ms/step - loss: 0.6093 - accuracy: 1.0000 - val_loss: 1.4987 - val_accuracy: 0.7500
Epoch 7/100
3/3 [==============================] - 0s 26ms/step - loss: 0.5099 - accuracy: 1.0000 - val_loss: 2.4889 - val_accuracy: 0.5000
Epoch 8/100
3/3

### Put everything together

You have tested various models on your four chosen datasets. Which model shows the best performance? You need to write a simple report which should be max two pages about your trials.
 - Task 0: State three univariate time series datasets you chose for the tasks.
 - Task 1: Report the average test scores of four models on three datasets you chose. It would be four scores in total. Mark the best model in terms of the average test score. Briefly explain the structure you constructed. 
 - Task 2: Report test scores of three models on three datasets you chose. It would be nine scores in total. Report the rank of average accuracy scores of three models. Briefly explain two deep learning model structures you constructed to confirm that you correctly developed the models in the paper.
 - Task 3: Briefly explain which model you chose, how you improved it. Report test scores of the model on three datasets you chose.
 - Task 4: Explain your choice of dataset and (classifier, transformer) pair and the parameters you tried to optimize. Report the best score and estimator from the randomized search instance and the test score of the best model.  Compare your best score to the score from the same model with the default setting.
 - Task 5: Report the best model (ROCKET vs your deep learning model) in terms of the test score. Briefly explain how you handle the multivariate dataset for your two models.